# Monitoring Environmental Waste Utilization Scores
# A guide for generating EWU-Dashboards
**Sarah Schmidt & David Laner**  
*January 2023*

## 1. Configuration

### 1.1 Import packages

In [3]:
import pandas as pd
import numpy as np
import brightway2 as bw
import premise
from mycolorpy import colorlist as mcp
import string

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [4]:
# create a list with MS Excel column indices (A, B, ..., Z, AA, AB, ...)
# will be used for preparing calculations in MS Excel
alphabet = list(string.ascii_uppercase)
excel_cols=[]

for i in range(100):
    n=0
    j=i
    while j-len(alphabet)>=0:
        j=j-len(alphabet)
        n=n+1
    if n>0:
        col=alphabet[n-1]+alphabet[j]
    else:
        col=alphabet[j]
    excel_cols.append(col)

In [8]:
conversion_factors={'kg':1,
                    't':1e3,
                    'kt':1e6,
                    'g':1e-3,
                    'Mg':1e3,
                    'Gg':1e6}

### 1.2 Gather case study-specific information

In [56]:
# read various case study specific information from input data file (ExcelTool_GeneratorInput_Template.xlsx)
general_info=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='GeneralInformation', index_col=0)
general_info=general_info[general_info.columns[0]].to_dict()

Pathway_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_codes=Pathway_codes[Pathway_codes['Pathway tag'].notnull()]['Activity code'].to_list()
Pathway_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_names=Pathway_names[Pathway_names['Pathway tag'].notnull()]['Pathway tag'].to_list()

WasteGen_code=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
WasteGen_code=WasteGen_code[WasteGen_code['Waste generation'].notnull()]['Activity code'].iloc[0]

act_codes=[WasteGen_code,*Pathway_codes]
act_names=[general_info['Acronym'],*Pathway_names]

# foreground system
foreground_system_codes=activity_df[activity_df['Foreground'].notnull()]['Activity code'].to_list()

# number of foreground system scenarios
n_scenarios=general_info['Maximum number of foreground system scenarios']

# materials for calculation of the environmental impact of materials
material_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_names=material_names['Material Name'].to_list()
material_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_codes=material_codes['Material Code'].to_list()

waste_utilization_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
waste_utilization_codes=waste_utilization_codes[pd.isnull(waste_utilization_codes['Waste utilization'])==False]['Activity code'].to_list()

# activity tags for contribution analysis
activity_tags=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
activity_tags=activity_tags['Activity tag'].to_list()
activity_tags=[tag for tag in activity_tags if pd.isnull(tag)==False]
activity_tags=list(set(activity_tags))
activity_tags.append('Others')

# waste quantity in kg
WQ_kg=general_info['Waste quantity']*conversion_factors[general_info['Unit']]

### 1.3 Setup and import of databases

#### 1.3.1 Setup

In [11]:
# create a new project or open an existing project
bw.projects.set_current(general_info['Project name'])

In [12]:
# creates the database "biosphere 3"
bw.bw2setup() 
biosphere = bw.Database("biosphere3")

Biosphere database already present!!! No setup is needed


#### 1.3.2 Ecoinvent

In [13]:
# import of the ecoinvent database
db_default_name=general_info['Database name']+'_default'
if db_default_name in bw.databases:
    print("Database has already been imported.")
    eidb_default = bw.Database(db_default_name)
else:
    # mind that the ecoinvent file must be unzipped; then: path to the datasets subfolder
    fpeidbcut = r"{}".format(general_info['Database file path'])
    # the "r" makes sure that the path is read as a string - especially useful when you have spaces in your string
    eidbcut = bw.SingleOutputEcospold2Importer(fpeidbcut, general_info['Database name']+'_default')
    eidbcut
    eidbcut.apply_strategies()
    eidbcut.statistics()
    eidb_default=eidbcut.write_database()

Database has already been imported.


In [14]:
# copy of the unmodified version of the ecoinvent database
if general_info['Database name'] in bw.databases:
    print("Database has already been imported.")
else:
    eidb_default.copy(general_info['Database name'])
eidb = bw.Database(general_info['Database name'])

Database has already been imported.


#### 1.3.3 Prospective Databases (premise)

In [15]:
# gather information which prospective scenarios shall be created
premise_scenarios=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                                skiprows=3, nrows=8)
premise_update=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                                index_col=0, skiprows=15, nrows=9, usecols='A:B')
premise_update=premise_update[premise_update['Update']=='yes']

In [17]:
# read encryption key 
# (to be requested from the premise library maintainers if you want ot use default scenarios included in `premise`)
encryption_key=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='ProspectiveScenarios', 
                         usecols = "B", header = 0, nrows=0).columns[0]

In [21]:
if general_info['Generate background system scenarios']=='yes':
    if general_info ['Type of background system scenarios']=='premise':
        
        premise_scenarios_dictlist=[]
        premise_scenario_names_list=[]

        for i in premise_scenarios.index:
            for c in premise_scenarios.columns[3:]:
                if premise_scenarios.loc[i,c]=='x':
                    scenario_dict={"model":premise_scenarios.loc[i,'IAM'], 
                                   "pathway":premise_scenarios.loc[i,'SSP']+'-'+premise_scenarios.loc[i,'RCP'], 
                                   "year":c}
                    scenario_name=premise_scenarios.loc[i,'IAM']+str(c)+'_'+premise_scenarios.loc[i,'SSP']+'-'+premise_scenarios.loc[i,'RCP']
                    premise_scenarios_dictlist.append(scenario_dict)
                    premise_scenario_names_list.append(scenario_name)
       
                    if scenario_name not in bw.databases:
                        ndb = premise.NewDatabase(
                            scenarios=[scenario_name],
                            source_db=eidb_default.name, # name of the database in the BW2 project. Must be a string.
                            source_version=general_info['Database version'], # version of ecoinvent. Can be "3.5", "3.6", "3.7" or "3.8". Must be a string.
                            key=encryption_key,# <-- decryption key
                            quiet=True
                            # to be requested from the library maintainers if you want ot use default scenarios included in `premise`
                            )


                        if len(premise_update)==8:
                            ndb.update_all()
                        else:
                            if 'Electricity' in premise_update.index:
                                ndb.update_electricity()
                            if 'Cement' in premise_update.index:
                                ndb.update_cement()
                            if 'Steel' in premise_update.index:
                                ndb.update_steel()
                            if 'Fuels' in premise_update.index:
                                ndb.update_fuels()
                            if 'Cars' in premise_update.index:
                                ndb.update_cars()
                            if 'Trucks' in premise_update.index:
                                ndb.update_trucks()
                            if 'Two wheelers' in premise_update.index:
                                ndb.update_two_wheelers()
                            if 'Buses' in premise_update.index:
                                ndb.update_buses()

                        ndb.write_db_to_brightway(name=[scenario_name])    

#### 1.3.4 Manual Background Scenarios

In [22]:
background_scenarios=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', 
                                   sheet_name='BackgroundScenarios')
background_scenarios

,background scenario,database,input,input code,input unit,input location / biosphere category,activity,activity code,activity location,amount,type


In [23]:
if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='manual') or (general_info ['Type of background system scenarios']=='manual + premise'):
        
        background_scenario_names_list=[]
        
        for i in background_scenarios.index:
            db_name=background_scenarios.loc[i,'database']+'_'+background_scenarios.loc[i,'background scenario']
            if db_name not in bw.databases:
                db=bw.Database(background_scenarios.loc[i,'database'])
                db.copy(db_name)
            else:
                print("Database has already been imported.")
            bsdb=bw.Database(db_name)
            
            act=bsdb.get(background_scenarios.loc[i,'activity code'])
            exc=[exc for exc in act.exchanges() if exc.input.as_dict()['code']==background_scenarios.loc[i,'input code']][0]
            
            exc['amount']=background_scenarios.loc[i,'amount']
            exc.save()
            
            if db_name not in background_scenario_names_list:
                background_scenario_names_list.append(db_name)

#### 1.3.5 Overview of databases to be included in the EWU-Dashboard

In [24]:
dbs=[eidb]

db_names_dict={}
db_names_dict['default']=eidb.name
n=1

if general_info['Generate background system scenarios']=='yes':
    if general_info ['Type of background system scenarios']=='premise':
        for scenario in premise_scenario_names_list:
            dbs.append(bw.Database(scenario))
            db='db'+str(n)
            db_names_dict[db]=scenario
            n=n+1

if general_info['Generate background system scenarios']=='yes':
    if (general_info ['Type of background system scenarios']=='manual') or (general_info['Type of background system scenarios']=='manual + premise'):
        for scenario in background_scenario_names_list:
            dbs.append(bw.Database(scenario))
            db='db'+str(n)
            db_names_dict[db]=scenario
            n=n+1
            
db_names=[db for db in db_names_dict.keys()]

### 1.4 LCIA Methods, Normalization Factors & Weighting Factors

**Methods**

In [65]:
LCIAmethod_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='LCIA_Methods')
LCIA_method_names=LCIA_method_df['Acronym'].to_list()

In [67]:
LCIA_methods=[]
for i in LCIAmethod_df.index:
    method=[m for m in bw.methods if m[0]==LCIAmethod_df.loc[i, 'Method_Part1'] and
                                     m[1]==LCIAmethod_df.loc[i, 'Method_Part2'] and
                                     m[2]==LCIAmethod_df.loc[i, 'Method_Part3']][0]
    LCIA_methods.append(method)

In [70]:
LCIAmethod_sheet_name='LCIA_Methods'

In [69]:
LCIAmethod_df=LCIAmethod_df.set_index('Acronym', drop=True)

**Normalization Factors**

In [68]:
nfs=LCIAmethod_df['Normalization Factor'].to_list()

**Weighting Factors**

In [87]:
Weighting_sheet_name='Weighting'

In [88]:
Weighting=pd.read_excel("ExcelTool_GeneratorInput_Template.xlsx", sheet_name="Weighting", index_col=0)

## 2. Calculations

### 2.1 Life Cycle Inventory

#### 2.1.1 Activities

In [25]:
activity_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')

In [26]:
for db in dbs:
    print(db.name)
    for i in activity_df.index:
        if len([act for act in db if act['code']==activity_df.loc[i,'Activity code']])==0:
            activity = db.new_activity(code = activity_df.loc[i,'Activity code'], name = activity_df.loc[i,'Activity name'], unit = activity_df.loc[i,'Unit'], location = activity_df.loc[i,'Location'])
            activity.save()
        else:
            activity=[act for act in db if act['code']==activity_df.loc[i,'Activity code']][0]
            if activity['name']!=activity_df.loc[i,'Activity name']:
                print("Error: Activity name", activity['name'], activity_df.loc[i,'Activity name'])
            if activity['location']!=activity_df.loc[i,'Location']:
                print("Error: Activity location", activity['location'], activity_df.loc[i,'Location'])
            if activity['unit']!=activity_df.loc[i,'Unit']:
                print("Error: Activity unit", activity['unit'], activity_df.loc[i,'Unit'])

ecoinvent 3.7.1_cutoff_ecoSpold02
image2030_SSP2-RCP19
image2050_SSP2-RCP19


#### 2.2.2 Exchanges

In [27]:
exchanges_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Exchanges')

In [28]:
for db in dbs:
    print(db.name)
    for actcode in exchanges_df['activity code'].unique():
        act_exchanges_df=exchanges_df[exchanges_df['activity code']==actcode]
        act=[act for act in db if act['code']==actcode][0]
        act.exchanges().delete()
        for i in act_exchanges_df.index:
            if (act_exchanges_df.loc[i,'type']=='technosphere') or (act_exchanges_df.loc[i,'type']=='production'):
                exc_input=[act for act in db if act['code']==act_exchanges_df.loc[i,'input code']][0]
            if act_exchanges_df.loc[i,'type']=='biosphere':
                exc_input=[act for act in biosphere if act['code']==act_exchanges_df.loc[i,'input code']][0]
            act.new_exchange(input = exc_input.key, amount = act_exchanges_df.loc[i,'amount'], 
                                 unit = act_exchanges_df.loc[i,'input unit'], type = act_exchanges_df.loc[i,'type']).save() 
            act.save()             

            exc=[exc for exc in act.exchanges() if exc['input']==exc_input.key][0]

            #import material flow tag
            if (exc_input['code'] in activity_df['Activity code']) & (exc_input['unit'] == 'kilogram'):
                exc['tag']='material flow'
                exc.save()
                act.save()

ecoinvent 3.7.1_cutoff_ecoSpold02
image2030_SSP2-RCP19
image2050_SSP2-RCP19


### 2.2 Modular Life Cycle Assessment

In [29]:
activity_df=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')

In [32]:
foreground_system=[]
for actcode in foreground_system_codes:
    foreground_system.append(eidb.get(actcode))

In [33]:
foreground_system_groups={}
for ac,actcode in enumerate(foreground_system_codes):
    foreground_system_groups[actcode]=ac

#### 2.2.1 LCI to DataFrame

In [40]:
LCI_df=pd.DataFrame()
i=0

for act in foreground_system:   
    act_name=act.as_dict()['name']
    act_code=act.as_dict()['code']
    act_unit=act.as_dict()['unit']
    act_location=act.as_dict()['location']
    #production exchange
    LCI_df.loc[i,'input']=act_name
    LCI_df.loc[i,'input code']=act_code
    LCI_df.loc[i,'input unit']=act_unit
    LCI_df.loc[i,'input location']=act_location
    LCI_df.loc[i,'activity']=act_name
    LCI_df.loc[i,'activity code']=act_code
    LCI_df.loc[i,'activity location']=act_location
    if len([exc for exc in eidb.get(act_code).production()]) >0:
        LCI_df.loc[i,'amount']=[exc.amount for exc in eidb.get(act_code).production()][0]
    else:
        LCI_df.loc[i,'amount']=1
    LCI_df.loc[i,'type']='production'
    LCI_df.loc[i,'material flow']=False
    i=i+1

    #technosphere exchanges
    for exc in act.technosphere():
        LCI_df.loc[i,'input']=exc.input.as_dict()['name']
        LCI_df.loc[i,'input code']=exc.input.as_dict()['code']
        LCI_df.loc[i,'input unit']=exc.input.as_dict()['unit']
        LCI_df.loc[i,'input location']=exc.input.as_dict()['location']
        LCI_df.loc[i,'activity']=act_name
        LCI_df.loc[i,'activity code']=act_code
        LCI_df.loc[i,'activity location']=act_location
        LCI_df.loc[i,'amount']=exc.amount
        LCI_df.loc[i,'type']='technosphere'
        if exc.input.as_dict()['code'] in foreground_system_codes:
            LCI_df.loc[i,'material flow']=True
        else:
            LCI_df.loc[i,'material flow']=False
        i=i+1
        
    #biosphere exchanges
    for exc in act.biosphere():
        LCI_df.loc[i,'input']=exc.input.as_dict()['name']
        LCI_df.loc[i,'input code']=exc.input.as_dict()['code']
        LCI_df.loc[i,'input unit']=exc.input.as_dict()['unit']
        LCI_df.loc[i,'input location']=str(exc.input.as_dict()['categories'])
        LCI_df.loc[i,'activity']=act_name
        LCI_df.loc[i,'activity code']=act_code
        LCI_df.loc[i,'activity location']=act_location
        LCI_df.loc[i,'amount']=exc.amount
        LCI_df.loc[i,'type']='biosphere'
        LCI_df.loc[i,'material flow']=False
        i=i+1

LCI_df

,input,input code,input unit,input location,activity,activity code,activity location,amount,type,material flow
0,biowaste generation,waste_generation,kilogram,DE,biowaste generation,waste_generation,DE,-1.000000e+00,production,False
1,home composting,PW1,kilogram,DE,biowaste generation,waste_generation,DE,-2.000000e-01,technosphere,True
2,industrial biowaste treatment,PW2,kilogram,DE,biowaste generation,waste_generation,DE,-8.000000e-01,technosphere,True
3,home composting,PW1,kilogram,DE,home composting,PW1,DE,-1.000000e+00,production,False
4,"home composting, apples",PW1_apples,kilogram,DE,home composting,PW1,DE,-8.000000e-01,technosphere,True
...,...,...,...,...,...,...,...,...,...,...
106,"Carbon dioxide, non-fossil",73ed05cc-9727-4abf-9516-4b5c0fe54a16,kilogram,"('air', 'urban air close to ground')","treatment of biowaste by anaerobic digestion, ...",anaerobic_digestion_bananas,DE,2.100000e-01,biosphere,False
107,Dinitrogen monoxide,6dc1b46f-ee89-4495-95c4-b8a637bcd6cb,kilogram,"('air', 'urban air close to ground')","treatment of biowaste by anaerobic digestion, ...",anaerobic_digestion_bananas,DE,3.300000e-05,biosphere,False
108,"Nitrogen, organic bound",a703733d-fabc-487b-826a-06c11ac4c0c6,kilogram,"('water',)","treatment of biowaste by anaerobic digestion, ...",anaerobic_digestion_bananas,DE,1.090000e-07,biosphere,False
109,Nitrate,5189de76-6bbb-44ba-8c42-5714f1b4371f,kilogram,"('water',)","treatment of biowaste by anaerobic digestion, ...",anaerobic_digestion_bananas,DE,2.970000e-06,biosphere,False


#### 2.2.2 Add parameters

In [34]:
# Excel sheet name (EWU-Dashboard)
params_sheet_name='Parameter'

In [38]:
scenarios=['Default']

for s in range(n_scenarios):
    scenarios.append('S'+str(s+1))

In [41]:
parameter_df=pd.DataFrame()
p=1

parameter_df.loc[0,'Parameter-ID']='P1'
parameter_df.loc[0,'Parameter']='Waste Quantity'
for s in range(n_scenarios):
    parameter_df.loc[0,'Parameter Value - S'+str(s+1)]=WQ_kg
parameter_df.loc[0,'Parameter Value - Default']=WQ_kg
parameter_df.loc[0,'Unit']='kilogram'

for i in LCI_df.index:
    if LCI_df.loc[i,'type']=='technosphere':
        param='P'+str(p+1)
        LCI_df.loc[i,'formula']=param
        parameter_df.loc[p,'Parameter-ID']=param
        parameter_df.loc[p,'Parameter']='FROM: '+LCI_df.loc[i,'input']+' ('+LCI_df.loc[i,'input location']+', '+LCI_df.loc[i,'input unit']+'), TO: '+LCI_df.loc[i,'activity']+'('+LCI_df.loc[i,'activity location']+')'
        parameter_df.loc[p,'Unit']=LCI_df.loc[i,'input unit']       
        for s in range(n_scenarios):
            parameter_df.loc[p,'Parameter Value - S'+str(s+1)]=LCI_df.loc[i,'amount']
        parameter_df.loc[p,'Parameter Value - Default']=LCI_df.loc[i,'amount']
        if LCI_df.loc[i,'material flow'] == True:
            parameter_df.loc[p, 'Group']=foreground_system_groups[LCI_df.loc[i,'activity code']]
        p=p+1
        
parameter_df

,Parameter-ID,Parameter,Parameter Value - S1,Parameter Value - S2,Parameter Value - S3,Parameter Value - S4,Parameter Value - S5,Parameter Value - Default,Unit,Group
0,P1,Waste Quantity,1.500000e+03,1.500000e+03,1.500000e+03,1.500000e+03,1.500000e+03,1.500000e+03,kilogram,NaN
1,P2,"FROM: home composting (DE, kilogram), TO: biow...",-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,kilogram,0.0
2,P3,"FROM: industrial biowaste treatment (DE, kilog...",-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,kilogram,0.0
3,P4,"FROM: home composting, apples (DE, kilogram), ...",-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,-8.000000e-01,kilogram,1.0
4,P5,"FROM: home composting, bananas (DE, kilogram),...",-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,-2.000000e-01,kilogram,1.0
5,P6,"FROM: treatment of biowaste, home composting, ...",-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,kilogram,2.0
6,P7,"FROM: treatment of biowaste, home composting, ...",-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,kilogram,3.0
7,P8,"FROM: industrial biowaste treatment, apples (D...",-6.000000e-01,-6.000000e-01,-6.000000e-01,-6.000000e-01,-6.000000e-01,-6.000000e-01,kilogram,4.0
8,P9,"FROM: industrial biowaste treatment, bananas (...",-4.000000e-01,-4.000000e-01,-4.000000e-01,-4.000000e-01,-4.000000e-01,-4.000000e-01,kilogram,4.0
9,P10,"FROM: treatment of biowaste, industrial compos...",-5.000000e-01,-5.000000e-01,-5.000000e-01,-5.000000e-01,-5.000000e-01,-5.000000e-01,kilogram,5.0


In [42]:
params_col_dict={}
for c,col in enumerate(parameter_df.columns):
    params_col_dict[col]=excel_cols[c]
    
params_row_dict={}
for p,param in enumerate(parameter_df['Parameter-ID']):
    params_row_dict[param]=p+2
#params_row_dict

#### 2.2.3 LCA calculations for unique exchanges

**Technosphere**

In [54]:
LCI_df_tech=LCI_df[LCI_df['type']=='technosphere']
unique_inputs_tech=LCI_df_tech['input code'].unique()
unique_activities_tech=LCI_df_tech['activity code'].unique()
CF_dict_techno={}

for d,db in enumerate(dbs):
    functional_units_tech=[]
    reference_actcodes_tech=[]

    for actcode in unique_inputs_tech:
        if actcode not in unique_activities_tech:
            reference_actcodes_tech.append(actcode)
            functional_unit={db.get(actcode):1}
            functional_units_tech.append(functional_unit)
        
    calculation_setup = {'inv': functional_units_tech, 'ia': LCIA_methods}
    bw.calculation_setups['excel tool'] = calculation_setup
    mlca = bw.MultiLCA('excel tool')
    
    CF_dict_techno_db={}

    for a,actcode in enumerate(reference_actcodes_tech):
        CF_dict_techno_flow={}
        for i,IC in enumerate(LCIA_method_names):
            CF_dict_techno_flow[IC]=mlca.results[a,i]
        CF_dict_techno_db[actcode]=CF_dict_techno_flow
    
    CF_dict_techno[db_names[d]]=CF_dict_techno_db

**Biosphere**

In [57]:
LCIA_fp=r"{}".format(general_info['LCIA implementation file path'])

In [58]:
LCIA=pd.read_excel(LCIA_fp,sheet_name='CFs')

LCI_df_bio=LCI_df[LCI_df['type']=='biosphere']
unique_inputs_bio=LCI_df_bio['input code'].unique()
unique_activities_bio=LCI_df_bio['activity code'].unique()

functional_units_bio=[]
reference_actcodes_bio=[]

for actcode in unique_inputs_bio:
    if actcode not in unique_activities_bio:
        reference_actcodes_bio.append(actcode)
        functional_unit={biosphere.get(actcode):1}
        functional_units_bio.append(functional_unit)
        
functional_units=[*functional_units_tech,*functional_units_bio]
reference_actcodes=[*reference_actcodes_tech,*reference_actcodes_bio]

CF_dfs={}
for i,IC in enumerate(LCIA_method_names):
    CF_dfs[IC]=LCIA[(LCIA['Method']==LCIA_methods[i][0])&(LCIA['Category']==LCIA_methods[i][1])&(LCIA['Indicator']==LCIA_methods[i][2])]
    
    
CF_dict_bio={}
for actcode in reference_actcodes_bio:
    bioflow=biosphere.get(actcode)
    bioflow_name=bioflow['name']
    bioflow_c0=bioflow['categories'][0]
    if len(bioflow['categories'])==2:
        bioflow_c1=bioflow['categories'][1]
    else:
        bioflow_c1='unspecified'
    CF_dict_bio_bioflow={}
    for IC in LCIA_method_names:
        CF_df=CF_dfs[IC]
        CF_df_filtered=CF_df[(CF_df['Name']==bioflow_name)&(CF_df['Compartment']==bioflow_c0)&(CF_df['Subcompartment']==bioflow_c1)]
        if len(CF_df_filtered)==0:
            CF=0
        else:
            CF=CF_df_filtered['CF'].values[0]
        CF_dict_bio_bioflow[IC]=CF
    CF_dict_bio[actcode]=CF_dict_bio_bioflow    

#### 2.2.4 Add characterization factors to LCI-DataFrame

In [59]:
LCI_df_sheet_names={}

for db in db_names:
    for s in scenarios:
        LCI_df_sheet_names[db+'_'+s]='LCI_'+db+'_'+s

In [60]:
LCI_dbs={}

for d,db in enumerate(dbs):
    for s in scenarios:
        LCI_df_db=LCI_df.copy()

        for i in LCI_df_db.index:
            input_code=LCI_df_db.loc[i,'input code']
            exc_type=LCI_df_db.loc[i,'type']
            if exc_type=='biosphere':
                for IC in LCIA_method_names:
                    LCI_df_db.loc[i,IC]=CF_dict_bio[input_code][IC]
            if exc_type=='technosphere':
                if input_code in reference_actcodes_tech:
                    for IC in LCIA_method_names:
                        LCI_df_db.loc[i,IC]=CF_dict_techno[db_names[d]][input_code][IC] 
                        
        LCI_df_db['formula excel']=LCI_df_db['formula']
        parameters=parameter_df['Parameter-ID']        
            
        # Excel formula
        for i in LCI_df_db.index:
            formula=LCI_df_db.loc[i,'formula']
            if type(formula)==str:
                formula_new=formula
                for p,param in enumerate(reversed(parameters)):
                    formula_new=formula_new.replace(param,params_sheet_name+'!'\
                                +params_col_dict['Parameter Value - '+s]+str(params_row_dict[param]))
                formula_new='='+formula_new
                LCI_df_db.loc[i,'formula excel']=formula_new
            else:
                LCI_df_db.loc[i,'formula excel']='='+excel_cols[LCI_df_db.columns.get_loc('amount')]\
                                                    +str(i+2)
        
        for i in LCI_df_db.index:
            if LCI_df_db.loc[i,'type']!='production':
                for IC in LCIA_method_names:
                    LCI_df_db.loc[i,IC+' Impact']='='+excel_cols[LCI_df_db.columns.get_loc('formula excel')]\
                                                        +str(i+2)+'*'\
                                                    +excel_cols[LCI_df_db.columns.get_loc(IC)]+str(i+2)
                    
        for actcode in LCI_df_db['activity code'].unique():
            sub_df=LCI_df_db[LCI_df_db['activity code']==actcode]
            for n,IC in enumerate(LCIA_method_names):
                formula='='
                col=excel_cols[LCI_df_db.columns.get_loc(IC+' Impact')]
                for i in sub_df.index[1:]:
                    ind=str(i+2)
                    formula=formula+'+'+col+ind
                LCI_df_db.loc[sub_df.index[0], IC+' Impact']=formula
                
        LCI_df_db_prod=LCI_df_db[LCI_df_db['type']=='production']
        prod_dict={}
        
        for i in LCI_df_db_prod.index:
            actcode=LCI_df_db_prod.loc[i,'activity code']
            prod_dict[actcode]=i+2
            
        for i in LCI_df_db.index:
            if LCI_df_db.loc[i,'type']!='production':
                input_code=LCI_df_db.loc[i,'input code']
                if input_code in prod_dict.keys():
                    for n,IC in enumerate(LCIA_method_names):
                        col=excel_cols[LCI_df_db.columns.get_loc(IC+' Impact')]
                        LCI_df_db.loc[i,IC]='='+col+str(prod_dict[input_code])+'*'\
                                               +excel_cols[LCI_df_db.columns.get_loc('formula excel')]\
                                               +str(prod_dict[input_code])
                    
        LCI_dbs[db_names[d]+'_'+s]=LCI_df_db

In [61]:
LCI_df_col_dict={}
for c,col in enumerate(LCI_df_db.columns):
    LCI_df_col_dict[col]=excel_cols[c]

#### 2.2.5 LCA Results

In [71]:
LCA_results_sheet_names={}

for db in db_names:
    for s in scenarios:
        LCA_results_sheet_names[db+'_'+s]='LCA_'+db+'_'+s

In [72]:
LCA_results={}

for db in db_names:
    for s in scenarios:
        LCA_results_db=pd.DataFrame()
        actcodes=act_codes
        actnames=act_names

        for a,actcode in enumerate(actcodes):
            LCA_results_db.loc[a,'Pathway']=actnames[a]
            for i,IC in enumerate(LCIA_method_names):
                col=IC+' Impact'
                LCA_results_db.loc[a,IC]='=LCI_'+db+'_'+s+'!'+LCI_df_col_dict[col]+str(prod_dict[actcode])\
                                         +'/LCIA_Methods!D'+str(i+2)

        LCA_results[db+'_'+s]=LCA_results_db

### 2.3 Perturbation Analysis

In [62]:
if general_info['Conduct perturbation analysis'] == 'yes':

    default_values={}
    for i in parameter_df.index:
        default_values[parameter_df.loc[i,'Parameter-ID']]=parameter_df.loc[i,'Parameter Value - Default']

    perturbation_runs={}
    for r in parameter_df.index:
        run='run'+str(r+1)
        run_params={}
        for i in parameter_df.index:
            if i!=r:
                run_params[parameter_df.loc[i,'Parameter-ID']]=parameter_df.loc[i,'Parameter Value - Default']
            else:
                run_params[parameter_df.loc[i,'Parameter-ID']]=parameter_df.loc[i,'Parameter Value - Default']*1.001
        perturbation_runs[run]=run_params

    relevant_indices_runs={}
    for run in perturbation_runs.keys():
        relevant_indices={}
        for i in LCI_df_db.index:
            formula=LCI_df_db.loc[i,'formula']
            if type(formula)==str:
                evaluated_formula=formula
                for p,param in enumerate(reversed(perturbation_runs[run].keys())):
                    evaluated_formula=evaluated_formula.replace(param,str(perturbation_runs[run][param]))
                evaluated_formula=float(evaluated_formula)
                if LCI_df_db.loc[i,'amount']!=evaluated_formula:
                    relevant_indices[i]=evaluated_formula
                    #print(run,i,evaluated_formula)
        relevant_indices_runs[run]=relevant_indices
    relevant_indices_runs

    exc_dict={}
    for i in LCI_df.index:
        act=eidb.get(LCI_df.loc[i,'activity code'])
        exchg=[exc for exc in act.exchanges() if exc.input.as_dict()['code'] ==LCI_df.loc[i,'input code']][0]
        exc_dict[i]=exchg
    exc_dict

    perturbation_df=pd.DataFrame()

    wg=eidb.get(WasteGen_code)
    for r,run in enumerate(perturbation_runs.keys()):
        print(run)
        if run == 'run1':
            calculation_setup = {'inv': [{wg:1.001}], 'ia': LCIA_methods}
            bw.calculation_setups['perturbation'] = calculation_setup
            mlca = bw.MultiLCA('perturbation')       
            perturbation_df.loc['P'+str(r+1),LCIA_method_names]=mlca.results

        else:
            for ind in relevant_indices_runs[run].keys():
                default_amount=LCI_df.loc[ind,'amount']
                exc_dict[ind]['amount']=relevant_indices_runs[run][ind]
                exc_dict[ind].save()

            #multi LCA
            calculation_setup = {'inv': [{wg:1}], 'ia': LCIA_methods}
            bw.calculation_setups['perturbation'] = calculation_setup
            mlca = bw.MultiLCA('perturbation')       
            perturbation_df.loc['P'+str(r+1),LCIA_method_names]=mlca.results

            for ind in relevant_indices_runs[run].keys():
                default_amount=LCI_df.loc[ind,'amount']
                exc_dict[ind]['amount']=default_amount
                exc_dict[ind].save()        

    # calculation of sensitivity ratios
    calculation_setup = {'inv': [{wg:1}], 'ia': LCIA_methods}
    bw.calculation_setups['perturbation'] = calculation_setup
    default_mlca = bw.MultiLCA('perturbation')       
    perturbation_df.loc['P'+str(r+1),LCIA_method_names]=default_mlca.results

    senstivity_ratio_df=pd.DataFrame()

    for i,ind in enumerate(perturbation_df.index):
        for c,col in enumerate(perturbation_df.columns):
            delta_rel_lca=(perturbation_df.loc[ind,col]-default_mlca.results[0][c])/default_mlca.results[0][c]
            delta_rel_param=1.001
            senstivity_ratio_df.loc[ind,col]=delta_rel_lca/delta_rel_param

    senstivity_ratio_df=senstivity_ratio_df.reset_index(drop=True)

    parameter_df=pd.concat([parameter_df, senstivity_ratio_df], axis=1)

### 2.4 Waste Composition

In [75]:
Waste_Comp_sheet_names={}

for s in scenarios:
    Waste_Comp_sheet_names[s]='Waste_Composition_'+s

In [76]:
Waste_Comp={}

for s in scenarios:
    Waste_Comp_S=pd.DataFrame()
    for a,actname in enumerate(Pathway_names):
        sub_df=LCI_df[LCI_df['activity code']==Pathway_codes[a]]
        for i in sub_df.index:
            for mat in material_codes:
                if mat in sub_df.loc[i,'input code']:
                    Waste_Comp_S.loc[mat,actname]='=ABS(LCI_default_'+s+'!'+LCI_df_col_dict['formula excel']+str(i+2)+')'    
    Waste_Comp_S=Waste_Comp_S.loc[material_codes]
    Waste_Comp[s]=Waste_Comp_S

In [77]:
Waste_Comp_col_dict={}
for c,col in enumerate(Waste_Comp_S.columns):
    Waste_Comp_col_dict[col]=excel_cols[c+1]

### 2.5 Waste Generation

In [78]:
Waste_Gen_sheet_names={}

for s in scenarios:
    Waste_Gen_sheet_names[s]='Waste_Generation_'+s

In [79]:
Waste_Gen={}

for s in scenarios:
    Waste_Gen_S=pd.DataFrame(index=act_names)
    name_code_dict={}
    for a,ac in enumerate(Pathway_codes):
        name_code_dict[Pathway_names[a]]=ac

    sub_df=LCI_df[LCI_df['activity code']==WasteGen_code]
    for i in sub_df.index:
        for a,ac in enumerate(Pathway_codes):
            if ac == sub_df.loc[i,'input code']:
                Waste_Gen_S.loc[Pathway_names[a],'relative']='=ABS(LCI_default_'+s+'!'\
                                                        +LCI_df_col_dict['formula excel']+str(i+2)+')'
                Waste_Gen_S.loc[Pathway_names[a],'absolute [kg]']='=ABS(LCI_default_'+s+'!'\
                                                +LCI_df_col_dict['formula excel']+str(i+2)+'*'\
                                                +"HLOOKUP(\"Parameter Value - "+s+"\",Parameter!C1:"\
                                                +excel_cols[n_scenarios+2]+"2,2,FALSE))"
    #Waste_Gen_S=Waste_Gen_S.sort_index()
    for c,col in enumerate(Waste_Gen_S.columns):
        Waste_Gen_S.loc[act_names[0],col]='=SUM('+excel_cols[c+1]+str(3)+':'+excel_cols[c+1]+str(len(Waste_Gen_S)+1)+')'
    
    Waste_Gen[s]=Waste_Gen_S

### 2.6 Original Environmental Value

#### 2.6.1 Materials

In [80]:
EV_df=pd.read_excel("ExcelTool_GeneratorInput_Template.xlsx", index_col=0, sheet_name='EnvironmentalValue')

In [81]:
OEV_material_sheet_names={}

for db in db_names:
    OEV_material_sheet_names[db]='OEV_materials_'+db

In [82]:
material_dict={}
material_dict_db={}

for d,db in enumerate(dbs):
    for i in EV_df.index:
        material_dict_db[i]=[act for act in db if act['name']==EV_df.loc[i,'Material activity name']
                            and act['unit']==EV_df.loc[i,'Material activity unit']
                            and act['location']==EV_df.loc[i,'Material activity location']][0]
    material_dict[db_names[d]]=material_dict_db

In [83]:
OEV_materials={}

for d,db in enumerate(db_names):
    functional_units=[]
    for act in material_dict[db].values():
        functional_units.append({act:1})
    functional_units

    calculation_setup = {'inv': functional_units, 'ia': LCIA_methods}

    bw.calculation_setups['materials'] = calculation_setup

    mlca = bw.MultiLCA('materials')

    OEV_material_norm=np.empty(np.shape(mlca.results))

    for f,fu in enumerate(functional_units):
        OEV_material_norm[f]=mlca.results[f]/nfs

    OEV_materials_db=pd.DataFrame(OEV_material_norm, 
                 index=EV_df.index,
                 columns=LCIA_method_names)

    OEV_materials[db]=OEV_materials_db

In [84]:
OEV_materials_col_dict={}
for c,col in enumerate(OEV_materials[db].columns):
    OEV_materials_col_dict[col]=excel_cols[c+1]

#### 2.6.2 Treatment Paths

In [85]:
OEV_Pathways_sheet_names={}

for db in db_names:
    for s in scenarios:
        OEV_Pathways_sheet_names[db+'_'+s]='OEV_Pathways_'+db+'_'+s

In [86]:
OEV_Pathways={}

for db in db_names:
    for s in scenarios:
        OEV_Pathways_db=pd.DataFrame(index=act_names)
        actnames=Pathway_names

        for Pathway in actnames:
            for n,IC in enumerate(LCIA_method_names):
                OEV_Pathways_db.loc[Pathway,IC]='=SUMPRODUCT('+OEV_material_sheet_names[db]+'!'+OEV_materials_col_dict[IC]+'2:'+\
                                    OEV_material_sheet_names[db]+'!'+OEV_materials_col_dict[IC]+str(len(EV_df.index)+1)+','+ \
                                    Waste_Comp_sheet_names[s]+'!'+Waste_Comp_col_dict[Pathway]+'2:'+\
                                    Waste_Comp_sheet_names[s]+'!'+Waste_Comp_col_dict[Pathway]+str(len(EV_df.index)+1) \
                                    +')'
    
        for i,IC in enumerate(LCIA_method_names):
            OEV_Pathways_db.loc[act_names[0],IC]='=SUMPRODUCT('+Waste_Gen_sheet_names[s]+'!B3:'+Waste_Gen_sheet_names[s]+'!B'+str(len(actnames)+2)+','\
                                            +excel_cols[i+1]+str(3)+':'+excel_cols[i+1]+str(len(actnames)+2)+')'

        OEV_Pathways[db+'_'+s]=OEV_Pathways_db

### 2.7 Environmental Waste Utilization

#### 2.7.1 Per Pathway

In [90]:
Indicator_sheet_names={}

for db in db_names:
    for s in scenarios:
        Indicator_sheet_names[db+'_'+s]='Indicator_'+db+'_'+s

In [91]:
Indicator={}

for db in db_names:
    for s in scenarios:
        Indicator_db=pd.DataFrame(index=OEV_Pathways_db.index,columns=OEV_Pathways_db.columns)

        for i,ind in enumerate(Indicator_db.index):
            for c,col in enumerate(Indicator_db.columns):
                Indicator_db.loc[ind,col]='=-'+LCA_results_sheet_names[db+'_'+s]+'!'\
                +excel_cols[c+1]+str(i+2)+'/'+OEV_Pathways_sheet_names[db+'_'+s]+'!'+excel_cols[c+1]+str(i+2)
        
        Indicator[db+'_'+s]=Indicator_db

#### 2.7.2 Aggregated

In [92]:
Weighted_Indicator_sheet_names={}

for db in db_names:
    for s in scenarios:
        Weighted_Indicator_sheet_names[db+'_'+s]='W_Ind_'+db+'_'+s

In [93]:
Weighted_Indicator={}

for db in db_names:
    for s in scenarios:
        Weighted_Indicator_db=pd.DataFrame()

        for a,act in enumerate(Indicator_db.index):
            for w,wm in enumerate(Weighting.columns):
                formula='='
                for i,IC in enumerate(LCIA_method_names):
                    formula=formula+'+'+Indicator_sheet_names[db+'_'+s]+'!'+excel_cols[i+1]+str(a+2)\
                            +'*'+Weighting_sheet_name+'!'+excel_cols[w+1]+str(i+2)
                Weighted_Indicator_db.loc[act,wm]=formula
        Weighted_Indicator[db+'_'+s]=Weighted_Indicator_db

### 2.8 Supply Chain

In [94]:
supply_chain_sheet_names={}

for s in scenarios:
    for act in act_names:
        supply_chain_sheet_names[s+'_'+act]='supply_chain_'+s+'_'+act

In [98]:
LCI=LCI_dbs[db_names[0]+'_'+scenarios[0]]

In [96]:
def supply_chain_per_Pathway(LCI,
                       activity_df,
                       supply_chain_df,
                       db, scenario, n_scenarios, 
                       amount_col=None,
                       excel_cols=excel_cols,
                       previous_input_code=None, 
                       previous_input_name=None,
                       first=True, x=None):
    if first == True:
        x=1
        amount_col=excel_cols[LCI.columns.get_loc('formula excel')]
        reference_amount="=VLOOKUP(\""+previous_input_name+"\",Waste_Generation_"+scenario+"!A1:C100,3,FALSE)*" \
                            +str(LCI[(LCI['type']=='production')&(LCI['input code']==previous_input_code)]['amount'].iloc[0])
        supply_chain_df.loc[previous_input_code,'amount']=reference_amount
        
    if previous_input_code not in LCI['activity code'].unique():
        return
    else:
        if x ==1000:
            return
        else:
            x=x+1
            ref=LCI[LCI['activity code']==previous_input_code]
            previous_index=supply_chain_df.index.get_loc(previous_input_code)+2
            production_index=ref[(ref['input code']==previous_input_code)&(ref['type']=='production')].index[0]+2
            for i in ref.index:
                if ref.loc[i,'type']!='production':
                    input_code=ref.loc[i,'input code']
                    if len(LCI[(LCI['input code']==input_code)&(LCI['type']=='production')])>0:
                        act_index=LCI[(LCI['input code']==input_code)&(LCI['type']=='production')].index[0]+2
                    else:
                        act_index=LCI[(LCI['input code']==input_code)].index[0]+2
                    if pd.isnull(supply_chain_df.loc[input_code,'amount'])==True:
                        supply_chain_df.loc[input_code,'amount']='=B'+str(previous_index)+'*'+'LCI_'+db+'_'+scenario+'!'\
                                                                +amount_col+str(i+2)+'*'+'LCI_'+db+'_'+scenario+'!'\
                                                                +amount_col+str(production_index)
                    else:
                        supply_chain_df.loc[input_code,'amount']=supply_chain_df.loc[input_code,'amount']+'+B'\
                                                                +str(previous_index)+'*'+'LCI_'+db+'_'+scenario+'!'\
                                                                +amount_col+str(i+2)+'*'+'LCI_'+db+'_'+scenario+'!'\
                                                                +amount_col+str(production_index)
                    previous_input_code=input_code
                    supply_chain_per_Pathway(LCI=LCI,
                                       activity_df=activity_df,
                                       supply_chain_df=supply_chain_df,
                                       db=db, scenario=scenario, n_scenarios=n_scenarios, 
                                       amount_col=amount_col,
                                       excel_cols=excel_cols,
                                       previous_input_code=previous_input_code, 
                                       first=False,x=x)

In [99]:
supply_chain={}

for scenario in scenarios:
    for a,actcode in enumerate(act_codes):
        #print(scenario+'_'+act_names[a], actcode)
        supply_chain_df=pd.DataFrame(index=LCI['input code'].unique())

        supply_chain_per_Pathway(LCI,
                       activity_df, 
                       supply_chain_df,
                       db, scenario, n_scenarios,
                       previous_input_code=actcode,
                       previous_input_name=act_names[a])

        supply_chain_df=supply_chain_df.replace(np.nan,0)

        supply_chain[scenario+'_'+act_names[a]]=supply_chain_df

### 2.9 Contribution Analysis

#### 2.9.1 Pathways

In [100]:
CA_Pathways_sheet_names={}

for db in db_names:
    for s in scenarios:
        CA_Pathways_sheet_names[db+'_'+s]='CA_Pathways_'+db+'_'+s

In [465]:
CA_Pathways={}
amount_col=excel_cols[LCI_df_db.columns.get_loc('formula excel')]

for db in db_names:
    for s in scenarios:
        CA_Pathways_db=pd.DataFrame(index=OEV_Pathways_db.index, columns=LCIA_method_names)

        for n,IC in enumerate(CA_Pathways_db.columns):
            for i,ind in enumerate(CA_Pathways_db.index):
                 CA_Pathways_db.loc[ind,IC]='='+LCA_results_sheet_names[db+'_'+s]+'!'+excel_cols[n+1]\
                                    +str(i+2)+'*'+Waste_Gen_sheet_names[s]+'!C'+str(i+2)

        CA_Pathways[db+'_'+s]=CA_Pathways_db

#### 2.9.2 Activities

In [101]:
CA_acts_sheet_names={}

for db in db_names:
    for s in scenarios:
        CA_acts_sheet_names[db+'_'+s]='CA_acts_'+db+'_'+s

In [102]:
# activity tags
activity_tag_dict={}

for tag in activity_df['Activity tag'].unique():
    if pd.isnull(tag)==False:
        tag_list=[]
        sub_df=activity_df[activity_df['Activity tag']==tag]
        for i in sub_df.index:
            sub_dict={}
            sub_dict['name']=activity_df.loc[i,'Activity name']
            sub_dict['unit']=activity_df.loc[i,'Unit']
            sub_dict['location']=activity_df.loc[i,'Location']
            tag_list.append(sub_dict)
        activity_tag_dict[tag]=tag_list

In [104]:
df=LCI_dbs['default_Default']
db='default'
scenario='Default'

In [105]:
for tag in activity_tag_dict.keys():
    for tag_act in activity_tag_dict[tag]:
        name=tag_act['name']
        location=tag_act['location']
        unit=tag_act['unit']
        for i in df.index:
            if (df.loc[i,'input']==name)&(df.loc[i,'input location']==location)&(df.loc[i,'input unit']==unit):
                df.loc[i,'activity tag']=tag

In [106]:
for db in db_names:
    for scenario in scenarios:
        for tag in activity_tag_dict.keys():
            for tag_act in activity_tag_dict[tag]:
                if len(df[(df['activity']==tag_act['name'])&(df['type']=='production')])>0:
                    for i in df[(df['activity']==tag_act['name'])&(df['type']=='production')].index:
                        sub_dict={}
                        sub=df[(df['activity']==tag_act['name'])&(df['activity location']==tag_act['location'])]
                        index=sub[(sub['activity tag'].isnull()==False)&(sub['type']=='production')].index[0]
                        sub_tags=sub[(sub['activity tag'].isnull()==False)&(sub['type']!='production')].index
                        tag_act['sub_tags']=sub_tags
                        ref_amount= LCI_df_col_dict['formula excel']+str(index+2)
                else:
                    index=df[(df['input']==tag_act['name'])].index[0]
                    ref_amount= LCI_df_col_dict['formula excel']+str(index+2)
                tag_act['index']=index
                tag_act['ref_amount']=ref_amount
                code=df.loc[index,'input code']
                tag_act['code']=code

In [107]:
CA_acts={}

for db in db_names:
    for scenario in scenarios:
        CA_acts_df=pd.DataFrame()
        for tag in activity_tag_dict:
            for i,IC in enumerate(LCIA_method_names):
                unit_impact='('
                
                for t,tag_act in enumerate(activity_tag_dict[tag]):
                    if t != 0:
                        unit_impact=unit_impact+'+'
                    unit_impact=unit_impact+supply_chain_sheet_names[scenario+'_'+act_names[0]]+'!B'\
                                    +str(supply_chain_df.index.get_loc(tag_act['code'])+2)+'*'+'('+'LCI_'+db+'_'+scenario+'!'+LCI_df_col_dict[IC+' Impact']\
                                    +str(tag_act['index']+2)
                    if 'sub_tags' in tag_act.keys():
                        for sub_tag in tag_act['sub_tags']:
                            unit_impact=unit_impact+'-LCI_'+db+'_'+scenario+'!'+LCI_df_col_dict[IC+' Impact']\
                                        +str(sub_tag+2)
                    unit_impact=unit_impact+')'+'/'+'LCI_'+db+'_'+scenario+'!'+tag_act['ref_amount']

                CA_acts_df.loc[tag,IC]='='+unit_impact+')'+'/LCIA_Methods!D'+str(i+2)
        CA_acts[db+'_'+scenario]=CA_acts_df

In [108]:
n_tags=len(activity_tag_dict.keys())

In [109]:
for db in db_names:
    for scenario in scenarios:
        CA_acts_df=CA_acts[db+'_'+scenario]
        for i,IC in enumerate(LCIA_method_names):
            col=excel_cols[i+1]
            reference_amount='ABS('+supply_chain_sheet_names[scenario+'_'+act_names[0]]+'!B'\
                                    +str(supply_chain_df.index.get_loc(WasteGen_code)+2)+')'
            CA_acts_df.loc['Others', IC]='=LCA_'+db+'_'+scenario+'!'+col+'2*'+reference_amount+'-SUM('+col+'2:'\
                                        +col+str(n_tags+1)+')'

### 2.10 EWU-Components: Treatment - Utilization - Material

#### 2.10.1 Per Impact Category

In [110]:
TUM_sheet_names={}

for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            TUM_sheet_names[db+'_'+s+'_'+TUM_type]='TUM_'+db+'_'+s+'_'+TUM_type

In [587]:
sc_indices={}
LCI_indices={}
for wuc in waste_utilization_codes:
    sc_indices[wuc]=str(supply_chain_df.index.get_loc(wuc)+2)
    LCI_indices[wuc]=str(LCI[LCI['input code']==wuc].index[0]+2)
sc_indices

{'911c4ec4da61447f2c8e05e76f556443': '20',
 '381392ac35ec7f41e20fae60c431dc52': '40'}

In [588]:
LCI_indices#[LCI['input code']==wuc].index[0]

{'911c4ec4da61447f2c8e05e76f556443': '29',
 '381392ac35ec7f41e20fae60c431dc52': '59'}

In [682]:
TUM={}

for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            TUM_df=pd.DataFrame(index=act_names, columns=LCIA_method_names)
            if TUM_type == 'Util':
                for act in TUM_df.index:
                    for i,IC in enumerate(LCIA_method_names):
                        formula='='
                        for wuc in waste_utilization_codes:
                            formula = formula + '+(supply_chain_'+scenario+'_'+act+'!B'+sc_indices[wuc]+'*'+'LCI_'+db+'_'+scenario+'!'+LCI_df_col_dict[IC+' Impact']\
                                                    +LCI_indices[wuc]+'/'+'LCI_'+db+'_'+scenario+'!AB'+LCI_indices[wuc]+')'                   
                        TUM_df.loc[act, IC]=formula+'/LCIA_Methods!D'+str(i+2)
                TUM[db+'_'+s+'_'+'Util']=TUM_df
            if TUM_type == 'Treatm':
                for a,act in enumerate(TUM_df.index):
                    for i,IC in enumerate(LCIA_method_names):
                        TUM_df.loc[act, IC]='=CA_Pathways_'+db+'_'+s+'!'+excel_cols[i+1]+str(a+2)+\
                                            '-TUM_'+db+'_'+s+'_'+'Util'+'!'+excel_cols[i+1]+str(a+2)
                TUM[db+'_'+s+'_'+'Treatm']=TUM_df
            if TUM_type == 'Mat':
                for a,act in enumerate(TUM_df.index):
                    for i,IC in enumerate(LCIA_method_names):
                        TUM_df.loc[act, IC]='=OEV_Pathways_'+db+'_'+s+'!'+excel_cols[i+1]+str(a+2)+"*VLOOKUP(\""+act+"\",Waste_Generation_"+scenario+"!A1:C100,3,FALSE)"
                TUM[db+'_'+s+'_'+'Mat']=TUM_df                


In [683]:
excel_cols[1]

'B'

In [685]:
TUM['db1'+'_'+'S1'+'_'+'Util']

,GW,OD,HTc,HTnc,PM,IR,POF,AC,TE,FE,ME,ET,LU,RDw,RDm,RDf
BWM,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AC29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AC59/LCI_db1_S5!AB59)/NormalizationFactors!D2,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AD29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AD59/LCI_db1_S5!AB59)/NormalizationFactors!D3,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AE29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AE59/LCI_db1_S5!AB59)/NormalizationFactors!D4,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AF29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AF59/LCI_db1_S5!AB59)/NormalizationFactors!D5,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AG29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AG59/LCI_db1_S5!AB59)/NormalizationFactors!D6,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AH29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AH59/LCI_db1_S5!AB59)/NormalizationFactors!D7,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AI29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AI59/LCI_db1_S5!AB59)/NormalizationFactors!D8,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AJ29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AJ59/LCI_db1_S5!AB59)/NormalizationFactors!D9,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AK29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AK59/LCI_db1_S5!AB59)/NormalizationFactors!D10,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AL29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AL59/LCI_db1_S5!AB59)/NormalizationFactors!D11,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AM29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AM59/LCI_db1_S5!AB59)/NormalizationFactors!D12,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AN29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AN59/LCI_db1_S5!AB59)/NormalizationFactors!D13,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AO29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AO59/LCI_db1_S5!AB59)/NormalizationFactors!D14,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AP29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AP59/LCI_db1_S5!AB59)/NormalizationFactors!D15,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AQ29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AQ59/LCI_db1_S5!AB59)/NormalizationFactors!D16,=+(supply_chain_S5_BWM!B20*LCI_db1_S5!AR29/LCI_db1_S5!AB29)+(supply_chain_S5_BWM!B40*LCI_db1_S5!AR59/LCI_db1_S5!AB59)/NormalizationFactors!D17
TP1,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AC29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AC59/LCI_db1_S5!AB59)/NormalizationFactors!D2,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AD29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AD59/LCI_db1_S5!AB59)/NormalizationFactors!D3,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AE29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AE59/LCI_db1_S5!AB59)/NormalizationFactors!D4,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AF29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AF59/LCI_db1_S5!AB59)/NormalizationFactors!D5,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AG29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AG59/LCI_db1_S5!AB59)/NormalizationFactors!D6,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AH29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AH59/LCI_db1_S5!AB59)/NormalizationFactors!D7,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AI29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AI59/LCI_db1_S5!AB59)/NormalizationFactors!D8,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AJ29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AJ59/LCI_db1_S5!AB59)/NormalizationFactors!D9,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AK29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AK59/LCI_db1_S5!AB59)/NormalizationFactors!D10,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AL29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AL59/LCI_db1_S5!AB59)/NormalizationFactors!D11,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AM29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db1_S5!AM59/LCI_db1_S5!AB59)/NormalizationFactors!D12,=+(supply_chain_S5_TP1!B20*LCI_db1_S5!AN29/LCI_db1_S5!AB29)+(supply_chain_S5_TP1!B40*LCI_db

In [487]:
# calculate waste utilization per treatment path

In [488]:
#material
#OEV_Pathways_default_Default

In [489]:
eidb.get('Pathway1')

'home composting' (kilogram, DE, None)

In [490]:
waste

'Biowaste'

In [491]:
x

In [492]:
# material = material value
# utilization = supply chain * CF
# treatment = LCA_result - utilization

##### Weighted

In [633]:
Weighted_TUM_sheet_names={}

for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            Weighted_TUM_sheet_names[db+'_'+s+'_'+TUM_type]='WTUM_'+db+'_'+s+'_'+TUM_type
Weighted_TUM_sheet_names

{'default_Default_Treatm': 'WTUM_default_Default_Treatm',
 'default_Default_Util': 'WTUM_default_Default_Util',
 'default_Default_Mat': 'WTUM_default_Default_Mat',
 'default_S1_Treatm': 'WTUM_default_S1_Treatm',
 'default_S1_Util': 'WTUM_default_S1_Util',
 'default_S1_Mat': 'WTUM_default_S1_Mat',
 'default_S2_Treatm': 'WTUM_default_S2_Treatm',
 'default_S2_Util': 'WTUM_default_S2_Util',
 'default_S2_Mat': 'WTUM_default_S2_Mat',
 'default_S3_Treatm': 'WTUM_default_S3_Treatm',
 'default_S3_Util': 'WTUM_default_S3_Util',
 'default_S3_Mat': 'WTUM_default_S3_Mat',
 'default_S4_Treatm': 'WTUM_default_S4_Treatm',
 'default_S4_Util': 'WTUM_default_S4_Util',
 'default_S4_Mat': 'WTUM_default_S4_Mat',
 'default_S5_Treatm': 'WTUM_default_S5_Treatm',
 'default_S5_Util': 'WTUM_default_S5_Util',
 'default_S5_Mat': 'WTUM_default_S5_Mat',
 'db1_Default_Treatm': 'WTUM_db1_Default_Treatm',
 'db1_Default_Util': 'WTUM_db1_Default_Util',
 'db1_Default_Mat': 'WTUM_db1_Default_Mat',
 'db1_S1_Treatm': 'WTUM_db

In [649]:
Weighted_TUM={}

for db in db_names:
    for s in scenarios:
        for TUM_type in ['Treatm', 'Util', 'Mat']:
            Weighted_TUM_df=pd.DataFrame()

            for a,act in enumerate(TUM_df.index):
                for w,wm in enumerate(Weighting.columns):
                    formula='='
                    for i,IC in enumerate(LCIA_method_names):
                        formula=formula+'+'+TUM_sheet_names[db+'_'+s+'_'+TUM_type]+'!'+excel_cols[i+1]+str(a+2)\
                                +'*'+Weighting_sheet_name+'!'+excel_cols[w+1]+str(i+2)
                    Weighted_TUM_df.loc[act,wm]=formula
            Weighted_TUM[db+'_'+s+'_'+TUM_type]=Weighted_TUM_df

In [650]:
Weighted_TUM['default_Default_Treatm']

,Equal Weighting,Importance and Robustness (Sala et al. 2018),Distance-to-Target (Castellani et al. 2016),Planetary Boundaries (Bjorn & Hauschild 2015),Quality and Maturity (ILCD 2011),Global Warming,Ozone depletion,"Human toxicity, carcinogenic","Human toxicity, non-carcinogenic",Respiratory effects / Particulate Matter,Ionising radiation,Photochemical ozone formation,Acidification,Terrestrial eutrophication,Freshwater eutrophication,Marine eutrophication,Ecotoxicity (freshwater),Land use,"Resource depletion, water","Resource depletion, minerals and metals","Resource depletion, fossils"
BWM,=+TUM_default_Default_Treatm!B2*Weighting!B2+TUM_default_Default_Treatm!C2*Weighting!B3+TUM_default_Default_Treatm!D2*Weighting!B4+TUM_default_Default_Treatm!E2*Weighting!B5+TUM_default_Default_Treatm!F2*Weighting!B6+TUM_default_Default_Treatm!G2*Weighting!B7+TUM_default_Default_Treatm!H2*Weighting!B8+TUM_default_Default_Treatm!I2*Weighting!B9+TUM_default_Default_Treatm!J2*Weighting!B10+TUM_default_Default_Treatm!K2*Weighting!B11+TUM_default_Default_Treatm!L2*Weighting!B12+TUM_default_Default_Treatm!M2*Weighting!B13+TUM_default_Default_Treatm!N2*Weighting!B14+TUM_default_Default_Treatm!O2*Weighting!B15+TUM_default_Default_Treatm!P2*Weighting!B16+TUM_default_Default_Treatm!Q2*Weighting!B17,=+TUM_default_Default_Treatm!B2*Weighting!C2+TUM_default_Default_Treatm!C2*Weighting!C3+TUM_default_Default_Treatm!D2*Weighting!C4+TUM_default_Default_Treatm!E2*Weighting!C5+TUM_default_Default_Treatm!F2*Weighting!C6+TUM_default_Default_Treatm!G2*Weighting!C7+TUM_default_Default_Treatm!H2*Weighting!C8+TUM_default_Default_Treatm!I2*Weighting!C9+TUM_default_Default_Treatm!J2*Weighting!C10+TUM_default_Default_Treatm!K2*Weighting!C11+TUM_default_Default_Treatm!L2*Weighting!C12+TUM_default_Default_Treatm!M2*Weighting!C13+TUM_default_Default_Treatm!N2*Weighting!C14+TUM_default_Default_Treatm!O2*Weighting!C15+TUM_default_Default_Treatm!P2*Weighting!C16+TUM_default_Default_Treatm!Q2*Weighting!C17,=+TUM_default_Default_Treatm!B2*Weighting!D2+TUM_default_Default_Treatm!C2*Weighting!D3+TUM_default_Default_Treatm!D2*Weighting!D4+TUM_default_Default_Treatm!E2*Weighting!D5+TUM_default_Default_Treatm!F2*Weighting!D6+TUM_default_Default_Treatm!G2*Weighting!D7+TUM_default_Default_Treatm!H2*Weighting!D8+TUM_default_Default_Treatm!I2*Weighting!D9+TUM_default_Default_Treatm!J2*Weighting!D10+TUM_default_Default_Treatm!K2*Weighting!D11+TUM_default_Default_Treatm!L2*Weighting!D12+TUM_default_Default_Treatm!M2*Weighting!D13+TUM_default_Default_Treatm!N2*Weighting!D14+TUM_default_Default_Treatm!O2*Weighting!D15+TUM_default_Default_Treatm!P2*Weighting!D16+TUM_default_Default_Treatm!Q2*Weighting!D17,=+TUM_default_Default_Treatm!B2*Weighting!E2+TUM_default_Default_Treatm!C2*Weighting!E3+TUM_default_Default_Treatm!D2*Weighting!E4+TUM_default_Default_Treatm!E2*Weighting!E5+TUM_default_Default_Treatm!F2*Weighting!E6+TUM_default_Default_Treatm!G2*Weighting!E7+TUM_default_Default_Treatm!H2*Weighting!E8+TUM_default_Default_Treatm!I2*Weighting!E9+TUM_default_Default_Treatm!J2*Weighting!E10+TUM_default_Default_Treatm!K2*Weighting!E11+TUM_default_Default_Treatm!L2*Weighting!E12+TUM_default_Default_Treatm!M2*Weighting!E13+TUM_default_Default_Treatm!N2*Weighting!E14+TUM_default_Default_Treatm!O2*Weighting!E15+TUM_default_Default_Treatm!P2*Weighting!E16+TUM_default_Default_Treatm!Q2*Weighting!E17,=+TUM_default_Default_Treatm!B2*Weighting!F2+TUM_default_Default_Treatm!C2*Weighting!F3+TUM_default_Default_Treatm!D2*Weighting!F4+TUM_default_Default_Treatm!E2*Weighting!F5+TUM_default_Default_Treatm!F2*Weighting!F6+TUM_default_Default_Treatm!G2*Weighting!F7+TUM_default_Default_Treatm!H2*Weighting!F8+TUM_default_Default_Treatm!I2*Weighting!F9+TUM_default_Default_Treatm!J2*Weighting!F10+TUM_default_Default_Treatm!K2*Weighting!F11+TUM_default_Default_Treatm!L2*Weighting!F12+TUM_default_Default_Treatm!M2*Weighting!F13+TUM_default_Default_Treatm!N2*Weighting!F14+TUM_default_Default_Treatm!O2*Weighting!F15